In [ ]:
########################################################################################################################
# Filename: BPMLL_paragraph_classification.ipynb
#
# Purpose: Multi-label Text-categorization, using neural networks, for paragraph-level
#          data as part of STAT 6500 final project.

# Author(s): Bobby (Robert) Lumpkin, Yue Li
#
# Library Dependencies: numpy, pandas, tensorflow, bpmll
########################################################################################################################

# Text Categorization With Neural Networks 

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from bpmll import bp_mll_loss
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sys
sys.path.append('../ThresholdFunctionLearning')    ## Append path to the ThresholdFunctionLearning directory to the interpreters
                                                   ## search path
from threshold_learning import predict_test_labels_binary    ## Import the 'predict_test_labels_binary()' function from the 
                                                             ## threshold_learning library

## Load & Pre-Process the Data

In [2]:
## Load 'content_paragraphs_ready.csv' into a pandas dataframe
data_filepath = "..\..\dataset\content_paragraphs_ready.csv"
paragraph_data = pd.read_csv(data_filepath)
paragraph_data.head()
#os.getcwd()

,para_id,full_text,threats/impacts,responses/actions,severity,susceptibility,self-efficacy,external-efficacy,response efficacy,public health,...,prosper,preview,moor,coverag,glow,profil,clash,incumb,frequent,unfound
0,214236,MURPHY: Again Martha we are defacto staying at...,1,1,0,1,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,214232,"GOV. PHIL MURPHY, (D-NJ): Yes. Good to be back...",1,1,1,1,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,214266,"BEAUMONT (ON SCREEN UPPER LEFT - ""FRIDAY MARCH...",0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,214246,"But in the meantime, my message to Louisiana i...",1,1,1,0,1,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,214238,"MURPHY: Yeah listen, we had gotten another shi...",0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
## Keep only 'doc_id', the label columns, and the tf-idf columns
tfidf_colnames = list(paragraph_data.columns[25:])
label_columns = list(paragraph_data.columns[2:15])
cols_toKeep = ['doc_id']
cols_toKeep.extend(tfidf_colnames)
cols_toKeep.extend(label_columns)
paragraph_data = paragraph_data[cols_toKeep]
paragraph_data.head()

,doc_id,murphi,martha,defacto,stay,home,state,million,us,you�,...,susceptibility,self-efficacy,external-efficacy,response efficacy,public health,economy,education,political evaluation,racial conflict,international ralations/foreign policies
0,text1,1.684247,1.348455,2.161368,3.118616,3.016311,0.91833,1.207125,1.383217,1.763428,...,1,1,0,1,1,0,0,0,0,0
1,text2,1.684247,1.348455,0.000000,0.000000,0.000000,3.67332,0.000000,0.000000,1.763428,...,1,1,0,1,1,0,0,0,0,0
2,text3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0,0,1,0,1,0,0,0,0,0
3,text4,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0,1,0,1,1,0,0,0,0,0
4,text5,1.684247,1.348455,0.000000,0.000000,0.000000,0.00000,0.000000,1.383217,0.000000,...,0,0,1,0,1,0,0,0,0,0


In [4]:
## Define the X and Y train and test matrices
X = paragraph_data[tfidf_colnames].to_numpy().astype(float)
Y = paragraph_data[label_columns].to_numpy().astype(float)

tf.random.set_seed(123)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33)

In [5]:
## These are the rows that don't have any associated labels
Y_gz = Y > 0
no_labels_id = np.where(~Y_gz.any(axis=1))[0]
no_labels_id

array([ 38,  58,  61,  67,  83, 132, 139, 142, 143, 150, 151, 166, 167,
       180, 214, 221, 240, 279, 282, 283, 284, 285, 286, 287, 288, 289],
      dtype=int64)

## Define and Train a BP-MLL Network (Training Instances That Have Atleast One Label)

In [6]:
## Since some instances don't have any labels, remove them from the dataset for use with BP-MLL
atleast_one_label_ids = paragraph_data.index
atleast_one_label_ids = atleast_one_label_ids.difference(no_labels_id)
X_hasLabel = X[atleast_one_label_ids, ]
Y_hasLabel = Y[atleast_one_label_ids, ]
X_train_hasLabel, X_test_hasLabel, Y_train_hasLabel, Y_test_hasLabel = train_test_split(X_hasLabel, Y_hasLabel, test_size = 0.33)

In [7]:
Y_test_hasLabel

array([[1., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       ...,
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

In [8]:
## Compute the proportion of labels that are equal to one in the test_hasLabel set
## This tells us what the hamming loss would be, if we constantly predicted 0
## We can compare our observed hamming loss against this, since we have sparse labels
prop_one_bpmll = np.sum(Y_test_hasLabel == 1) / (Y_test_hasLabel.shape[0] * Y_test_hasLabel.shape[1])
prop_one_bpmll

0.26136363636363635

In [9]:
## Start by defining and compiling the bp-mll loss network 
num_labels = len(label_columns)

model_bpmll = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_labels, activation = 'sigmoid')
])

optim_func_bpmll = tf.keras.optimizers.Adagrad(
    learning_rate = 0.001, initial_accumulator_value = 0.1, epsilon = 1e-07,
    name = 'Adagrad')

model_bpmll.compile(optimizer = optim_func_bpmll,
              loss = bp_mll_loss,
              )

In [10]:
model_bpmll.fit(X_train_hasLabel, Y_train_hasLabel, epochs = 500)

Epoch 1/500
6/6 [==============================] - 1s 2ms/step - loss: 0.9793
Epoch 2/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9795
Epoch 3/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9797
Epoch 4/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9818
Epoch 5/500
6/6 [==============================] - 0s 1ms/step - loss: 0.9800
Epoch 6/500
6/6 [==============================] - 0s 1ms/step - loss: 0.9755
Epoch 7/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9844
Epoch 8/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9828
Epoch 9/500
6/6 [==============================] - 0s 1ms/step - loss: 0.9789
Epoch 10/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9786
Epoch 11/500
6/6 [==============================] - 0s 2ms/step - loss: 0.9788
Epoch 12/500
6/6 [==============================] - 0s 1ms/step - loss: 0.9764
Epoch 13/500
6/6 [==============================] - 0s 1ms/st

### Compare Performance of BP-MLL Loss Using Different Threshold Function Approaches 

In [11]:
## Using a constant 0.5 threshold function, get the hamming loss for the trained network on the test set
predictions = model_bpmll.predict(X_test_hasLabel)
predictions_binary = model_bpmll.predict(X_test_hasLabel)
for i in range(Y_test_hasLabel.shape[0]):
    for j in range(Y_test_hasLabel.shape[1]):
        if predictions_binary[i, j] > 0.5:
            predictions_binary[i, j] = 1
        else:
            predictions_binary[i, j] = 0

# Get the hamming loss
metrics.hamming_loss(Y_test_hasLabel, predictions_binary)

0.30506993006993005

In [12]:
## Learn a Threshold Function
Y_train_pred = model_bpmll.predict(X_train_hasLabel)
Y_test_pred = model_bpmll.predict(X_test_hasLabel)
t_range = (0, 1)

test_labels_binary, threshold_function = predict_test_labels_binary(Y_train_pred, Y_train_hasLabel, Y_test_pred, t_range)
metrics.hamming_loss(Y_test_hasLabel, test_labels_binary)

0.20104895104895104

## Define and Train a Cross-Entropy Loss Network (Training Instances That Have Atleast One Label)

In [13]:
## Start with standard cross-entropy loss (bpmll used later)

model_ce_hasLabel = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_labels, activation = 'sigmoid')
])

optim_func_ce = tf.keras.optimizers.Adagrad(
    learning_rate = 0.001, initial_accumulator_value = 0.1, epsilon = 1e-07,
    name = 'Adagrad')

model_ce_hasLabel.compile(optimizer = optim_func_ce,
              loss = 'categorical_crossentropy',
              )

In [14]:
model_ce_hasLabel.fit(X_train_hasLabel, Y_train_hasLabel, epochs = 500)

Epoch 1/500
6/6 [==============================] - 0s 2ms/step - loss: 8.3696
Epoch 2/500
6/6 [==============================] - 0s 1ms/step - loss: 8.9561
Epoch 3/500
6/6 [==============================] - 0s 1ms/step - loss: 8.5743
Epoch 4/500
6/6 [==============================] - 0s 1ms/step - loss: 8.3959
Epoch 5/500
6/6 [==============================] - 0s 1ms/step - loss: 8.3762
Epoch 6/500
6/6 [==============================] - 0s 2ms/step - loss: 8.3423
Epoch 7/500
6/6 [==============================] - 0s 2ms/step - loss: 8.3446
Epoch 8/500
6/6 [==============================] - 0s 2ms/step - loss: 8.2525
Epoch 9/500
6/6 [==============================] - 0s 2ms/step - loss: 8.1520
Epoch 10/500
6/6 [==============================] - 0s 2ms/step - loss: 8.2999
Epoch 11/500
6/6 [==============================] - 0s 2ms/step - loss: 8.4940
Epoch 12/500
6/6 [==============================] - 0s 3ms/step - loss: 7.9969
Epoch 13/500
6/6 [==============================] - 0s 2ms/st

### Compare Performance of CE Loss Using Different Threshold Function Approaches

In [15]:
## Using a constant 0.5 threshold function, get the hamming loss for the trained network on the test set
predictions = model_ce_hasLabel.predict(X_test_hasLabel)
predictions_binary = model_ce_hasLabel.predict(X_test_hasLabel)
for i in range(Y_test_hasLabel.shape[0]):
    for j in range(Y_test_hasLabel.shape[1]):
        if predictions_binary[i, j] > 0.5:
            predictions_binary[i, j] = 1
        else:
            predictions_binary[i, j] = 0

# Get the hamming loss
metrics.hamming_loss(Y_test_hasLabel, predictions_binary)

0.638986013986014

In [16]:
## Learn a Threshold Function
Y_train_pred = model_ce_hasLabel.predict(X_train_hasLabel)
Y_test_pred = model_ce_hasLabel.predict(X_test_hasLabel)
t_range = (0, 1)

test_labels_binary, threshold_function = predict_test_labels_binary(Y_train_pred, Y_train_hasLabel, Y_test_pred, t_range)
metrics.hamming_loss(Y_test_hasLabel, test_labels_binary)

0.2770979020979021

## Define and Train a Cross-Entropy Loss Network (All Training Instances)

In [17]:
## Compute the proportion of labels that are equal to one in the test set
## This tells us what the hamming loss would be, if we constantly predicted 0
## We can compare our observed hamming loss against this, since we have sparse labels
prop_one = np.sum(Y_test == 1) / (Y_test.shape[0] * Y_test.shape[1])
prop_one

0.2283653846153846

In [18]:
## Start with standard cross-entropy loss (bpmll used later)

model_ce = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_labels, activation = 'sigmoid')
])

optim_func_ce = tf.keras.optimizers.Adagrad(
    learning_rate = 0.001, initial_accumulator_value = 0.1, epsilon = 1e-07,
    name = 'Adagrad')

model_ce.compile(optimizer = optim_func_ce,
              loss = 'categorical_crossentropy',
              )

In [19]:
model_ce.fit(X_train, Y_train, epochs = 500)

Epoch 1/500
7/7 [==============================] - 0s 1ms/step - loss: 8.3290
Epoch 2/500
7/7 [==============================] - 0s 2ms/step - loss: 8.1455
Epoch 3/500
7/7 [==============================] - 0s 1ms/step - loss: 7.9106
Epoch 4/500
7/7 [==============================] - 0s 1ms/step - loss: 7.6472
Epoch 5/500
7/7 [==============================] - 0s 1ms/step - loss: 8.0032
Epoch 6/500
7/7 [==============================] - 0s 1ms/step - loss: 7.9449
Epoch 7/500
7/7 [==============================] - 0s 1ms/step - loss: 7.9886
Epoch 8/500
7/7 [==============================] - 0s 1ms/step - loss: 7.8912
Epoch 9/500
7/7 [==============================] - 0s 1ms/step - loss: 7.7813
Epoch 10/500
7/7 [==============================] - 0s 1ms/step - loss: 8.0440
Epoch 11/500
7/7 [==============================] - 0s 2ms/step - loss: 7.9262
Epoch 12/500
7/7 [==============================] - 0s 1ms/step - loss: 8.0621
Epoch 13/500
7/7 [==============================] - 0s 1ms/st

### Compare Performance of CE Loss Using Different Threshold Function Approaches

In [20]:
## Using a constant 0.5 threshold function, get the hamming loss for the trained network on the test set
predictions = model_ce.predict(X_test)
predictions_binary = model_ce.predict(X_test)
for i in range(Y_test.shape[0]):
    for j in range(Y_test.shape[1]):
        if predictions_binary[i, j] > 0.5:
            predictions_binary[i, j] = 1
        else:
            predictions_binary[i, j] = 0

# Get the hamming loss
metrics.hamming_loss(Y_test, predictions_binary)

0.5825320512820513

In [21]:
## Learn a Threshold Function
Y_train_pred = model_ce.predict(X_train)
Y_test_pred = model_ce.predict(X_test)
t_range = (0, 1)

test_labels_binary, threshold_function = predict_test_labels_binary(Y_train_pred, Y_train, Y_test_pred, t_range)
metrics.hamming_loss(Y_test, test_labels_binary)

0.1778846153846154

In [24]:
## Compare predicted test labels for instance against true test labels in next cell
test_labels_binary[4,]

array([0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.])

In [25]:
## Compare true test labels for instane against predicted test labels in previous cell
Y_test[4,]

array([0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.])